In [80]:
import pandas as pd
import json
import requests
import re
from bs4 import BeautifulSoup

## Search city and food type
Get a list of businesses and their info:
- Name
- Address
- Star Rating
- Price Range
- Number of Reviews
- URL for business Yelp review page

In [456]:
#ENTER SEARCH TERMS BELOW:
cuisine_type = "Gastropubs"
location = "Albany, NY"


#Generate URL based on search terms
base_url = "https://www.yelp.com"
search_url = f"{base_url}/search?find_desc={cuisine_type}&find_loc={location}"

In [261]:
#request = requests.get(search_url)
#soup = BeautifulSoup(request.content,'html.parser')

In [457]:
star_container_class = "lemon--div__373c0__1mboc attribute__373c0__1hPI_ display--inline-block__373c0__2de_K u-space-r1 border-color--default__373c0__2oFDT"
price_range_class = "lemon--span__373c0__3997G text__373c0__2pB8f priceRange__373c0__2DY87 text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-bullet--after__373c0__1ZHaA"
review_count_class = "lemon--span__373c0__3997G text__373c0__2pB8f reviewCount__373c0__2r4xT text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_"
next_page_class = "lemon--a__373c0__IEZFH link__373c0__29943 next-link navigation-button__373c0__1D3Ug link-color--blue-dark__373c0__1mhJo link-size--default__373c0__1skgq"
search_result_class = "lemon--div__373c0__1mboc searchResult__373c0__1yggB border-color--default__373c0__2oFDT"

In [458]:
next_page_url = search_url
page_counter = 1
business_list = []

while next_page_url:
    request = requests.get(next_page_url)
    soup = BeautifulSoup(request.content,'html.parser')
    search_results = soup.findAll(class_=search_result_class)
    print(f"Page {page_counter}, {len(search_results)-1} results")
    for search_result in search_results:
        try:
            business_name_url = search_result.findAll('a', href=True)[1]
            business_info = {}
            business_info['url'] = f"https://www.yelp.com{business_name_url['href']}"
            business_info['name'] = business_name_url['name']
            business_info['address'] = search_result.find('address').text
            business_info['category'] = [category.text for category in search_result.findAll("a",attrs={"role":"link"})]
            business_info['star_rating'] = re.findall(r"[-+]?\d*\.\d+|\d+", 
                                                      search_result.find(
                                                          class_=star_container_class).find('div')['aria-label'] )[0]
            business_info['price_range'] = search_result.find(class_=price_range_class).text
            business_info['num_reviews'] = re.findall(r"[-+]?\d*\.\d+|\d+",
                                                      search_result.find(
                                                          class_=review_count_class).text )[0]
            business_info['image_shown'] = search_result.find('img')['src']
            business_list.append(business_info)
        except:
            None
            
    if soup.find(class_=next_page_class):
        next_page_url = base_url + soup.find(class_=next_page_class)['href']
        page_counter+=1
    else:
        break

len(business_list)

Page 1, 30 results
Page 2, 1 results


31

In [459]:
business_info_df = pd.DataFrame(business_list)
business_info_df.head()

,address,category,image_shown,name,num_reviews,price_range,star_rating,url
0,281 New Scotland Ave,"[American (Traditional), Beer Bar]",https://s3-media3.fl.yelpcdn.com/bphoto/H-GObC...,The Albany Ale & Oyster,240,$$,4.5,https://www.yelp.com/biz/the-albany-ale-and-oy...
1,1100 Madison Ave,"[American (New), Pubs]",https://s3-media3.fl.yelpcdn.com/bphoto/_q42sc...,Madison Pour House,125,$$,4.5,https://www.yelp.com/biz/madison-pour-house-al...
2,42 Howard St,"[Bars, American (Traditional), Gastropubs]",https://s3-media2.fl.yelpcdn.com/bphoto/rBAD9u...,The City Beer Hall,489,$$,4,https://www.yelp.com/biz/the-city-beer-hall-al...
3,942 Broadway,"[American (New), Pubs, Beer Gardens]",https://s3-media1.fl.yelpcdn.com/bphoto/6sjQSq...,Lost & Found,55,$$,4.5,https://www.yelp.com/biz/lost-and-found-albany...
4,79 N Pearl St,"[American (New), Music Venues, Venues & Event ...",https://s3-media4.fl.yelpcdn.com/bphoto/XjDwO1...,The Hollow Bar & Kitchen,304,$$,4,https://www.yelp.com/biz/the-hollow-bar-and-ki...


In [284]:
search_results[1].find('img')['src']

'https://s3-media4.fl.yelpcdn.com/bphoto/P2v0Wd5JNwJYAuEm36KE2w/ls.jpg'

## Get reviews for an individual business

In [463]:
index_num = 6
business_url = business_info_df.url[index_num]
business_name = business_info_df.name[index_num]
print(business_name)
print(business_url)

Druthers Brewing Company
https://www.yelp.com/biz/druthers-brewing-company-albany?osq=Gastropubs


In [464]:
next_page_class = "lemon--a__373c0__IEZFH link__373c0__29943 next-link navigation-button__373c0__1D3Ug link-color--blue-dark__373c0__1mhJo link-size--default__373c0__1skgq"
search_result_class = "lemon--li__373c0__1r9wz u-space-b3 u-padding-b3 border--bottom__373c0__uPbXS border-color--default__373c0__2oFDT"

star_container_class = "lemon--div__373c0__1mboc arrange-unit__373c0__1piwO border-color--default__373c0__2oFDT"
date_class = "lemon--span__373c0__3997G text__373c0__2pB8f text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_"
pic_class = "lemon--span__373c0__3997G photo-box-grid-item__373c0__2kFqV display--inline__373c0__1DbOG u-space-r2 u-space-b2 border-color--default__373c0__2oFDT"
pic_url_class = "lemon--img__373c0__3GQUb photo-box-img__373c0__O0tbt"
user_info_class = "lemon--div__373c0__1mboc user-passport-info border-color--default__373c0__2oFDT"

In [465]:
next_page_url = business_url
page_counter = 1
reviews_list = []

while next_page_url:
    request = requests.get(next_page_url)
    soup = BeautifulSoup(request.content,'html.parser')
    reviews = soup.findAll(class_=search_result_class)
    print(next_page_url)
    print(f"Page {page_counter}, {len(reviews)} reviews")
    review_counter=1
    for review in reviews:
        try:
            review_info = {}
            review_info["business_name"] = business_name
            review_info["date"] = review.find(class_=date_class).text
            review_info["review"] = review.find("span", attrs={"class": "lemon--span__373c0__3997G", "lang": "en"}).text
            review_info['star_rating'] = re.findall(r"[-+]?\d*\.\d+|\d+", 
                                                    review.find(
                                                          class_=star_container_class).find('div')['aria-label'] )[0]
            review_info["pic_count"] = len(review.find_all(class_=pic_class))
            review_info["pic_urls"] = [obj['src'] for obj in review.findAll(class_=pic_url_class)]
            try:
                review_info["user_id"] = review.find('a')['href'].split('userid=')[1]
            except:
                review_info["user_id"] = "FAIL"
            #for obj in review.find_all(class_=pic_url_class):
            #    review_info["pic_urls"].append(obj["src"])
            reviews_list.append(review_info)
        except:
            print("Skipped Review:", review_counter)

        review_counter+=1
            
    if soup.find("link", attrs={'rel':'next'}):
        next_page_url = soup.find("link", attrs={'rel':'next'})['href']
        page_counter+=1
    else:
        break
        
    if page_counter > 2:
        break

len(reviews_list)

https://www.yelp.com/biz/druthers-brewing-company-albany?osq=Gastropubs
Page 1, 20 reviews
https://www.yelp.com/biz/druthers-brewing-company-albany?osq=Gastropubs&start=20
Page 2, 20 reviews


40

In [468]:
reviews_df = pd.DataFrame(reviews_list)
reviews_df.head(10)

,business_name,date,pic_count,pic_urls,review,star_rating,user_id
0,Druthers Brewing Company,7/28/2019,3,[https://s3-media4.fl.yelpcdn.com/bphoto/stvcX...,"After a long day of drinking, exploring, seein...",5,hWDybu_KvYLSdEFzGrniTw
1,Druthers Brewing Company,7/14/2019,3,[https://s3-media3.fl.yelpcdn.com/bphoto/VEwcR...,Great American pub/brewery with awesome season...,5,NE0Y9JgKRGGfkq0r8rrWMA
2,Druthers Brewing Company,6/8/2019,1,[https://s3-media4.fl.yelpcdn.com/bphoto/61oph...,"Service was PERFECT! Our server was funny, kno...",5,q5l-dhIhv2gHtR42_KS3BA
3,Druthers Brewing Company,9/16/2019,2,[https://s3-media3.fl.yelpcdn.com/bphoto/vMhXV...,Definitely worth the trip - located on a dark ...,4,4nS_ParKta9NBuE3yl72qg
4,Druthers Brewing Company,4/23/2019,0,[],Update (4/23/19): the O.G. Veggie burger has r...,4,NwegX3oLEwXuOXI1d3KfRQ
5,Druthers Brewing Company,3/9/2019,3,[https://s3-media3.fl.yelpcdn.com/bphoto/DDvZZ...,Yelp hit it out of the park on this one! Was i...,5,_kP1PMm3G-ZFUZtBsP-WPA
6,Druthers Brewing Company,3/7/2019,3,[https://s3-media1.fl.yelpcdn.com/bphoto/Tg5Oj...,"i love druthers, how couldn't you? The comfort...",4,YX927Jezj85xgM_XX6bVxw
7,Druthers Brewing Company,2/10/2019,3,[https://s3-media2.fl.yelpcdn.com/bphoto/RTOS-...,So today we took a drive up to U - Albany to c...,5,Kc7IwXLrJVqAVMkX6pxWbA
8,Druthers Brewing Company,5/30/2019,0,[],Unsure how I have missed reviewing Druthers!! ...,4,FAIL
9,Druthers Brewing Company,8/15/2019,3,[https://s3-media4.fl.yelpcdn.com/bphoto/2dExk...,Druthers is the place to go for great food exc...,4,flXkJBf-clMeA_jC1sqGEg


In [416]:
#request = requests.get(business_url)
soup = BeautifulSoup(request.content,'html.parser')
reviews = soup.findAll(class_=search_result_class)

In [449]:
reviews[2].find('a')['href'].split('userid=')[1]

'q8rS7eD1nGJPY8zZyAMVWQ'

In [357]:
user_info_class = "lemon--div__373c0__1mboc user-passport-info border-color--default__373c0__2oFDT"

In [454]:
reviews[3].find('a')

<a class="lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--default__373c0__1skgq" href="/biz_photos/new-world-bistro-bar-albany-3?userid=hp5PVGYMd3MEefEcM426dQ" name="" rel="" target=""><p class="lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_ text-size--small__373c0__3SGMi"><span aria-hidden="true" class="lemon--span__373c0__3997G icon__373c0__ehCWV icon--18-camera" style="width:18px;height:18px"><svg class="icon_svg" height="18" viewbox="0 0 18 18" width="18" xmlns="http://www.w3.org/2000/svg"><path d="M15 15H3a2 2 0 0 1-2-2V6a2 2 0 0 1 2-2h2a2 2 0 0 1 2-2h4a2 2 0 0 1 2 2h2a2 2 0 0 1 2 2v7a2 2 0 0 1-2 2zM9 5a4 4 0 1 0 0 8 4 4 0 0 0 0-8zm0 6.5a2.5 2.5 0 1 1 0-5 2.5 2.5 0 0 1 0 5z"></path></svg></span> <!-- -->5 photos</p></a>

### Write dataframes to CSV files

In [469]:
business_info_df.to_csv('business_info.csv',index=False)
reviews_df.to_csv('reviews.csv',index=False)